In [1]:
import cv2 as cv
import numpy as np
import os
import keras
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [2]:
# import os
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir("/content/drive/My Drive/Colab Notebooks/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
data = []
label = []
cats = os.listdir("data/cat")
print(cats)
try:
    for cat in os.listdir("data/cat"):
        img = cv.imread("data/cat/" + cat, cv.IMREAD_GRAYSCALE)
        #print('Original Dimensions : ',img.shape)
        resized = cv.resize(img, (50, 50))
        data.append(resized)
        label.append(1)
    for dog in os.listdir("data/dog"):
        img = cv.imread("data/dog/" + dog, cv.IMREAD_GRAYSCALE)
        #print('Original Dimensions : ',img.shape)
        resized = cv.resize(img, (50, 50))
        data.append(resized)
        label.append(0)
except Exception as e:
    print(str(e))

pets = np.array(data)
labels = np.array(label)

np.save("pets",pets)
np.save("labels",labels)

['images (433).jpeg', 'images (24).jpeg', 'images (189).jpeg', 'images (287).jpeg', 'images (283).jpeg', 'images (276).jpeg', 'images (467).jpeg', 'images (375).jpeg', 'images (272).jpeg', 'images (149).jpeg', 'images (383).jpeg', 'images (274).jpeg', 'images (483).jpeg', 'images (29).jpeg', 'images (99).jpeg', 'images (308).jpeg', 'images (395).jpeg', 'images (469).jpeg', 'images (454).jpeg', 'images (154).jpeg', 'images (177).jpeg', 'images (493).jpeg', 'images (422).jpeg', 'images (94).jpeg', 'images (219).jpeg', 'images (200).jpeg', 'images (488).jpeg', 'images (505).jpeg', 'images (277).jpeg', 'images (523).jpeg', 'images (20).jpeg', 'images (544).jpeg', 'images (86).jpeg', 'images (21).jpeg', 'images (447).jpeg', 'images (547).jpeg', 'images (242).jpeg', 'images (473).jpeg', 'images (159).jpeg', 'image (8).jpeg', 'images (268).jpeg', 'images (542).jpeg', 'images (410).jpeg', 'images (402).jpeg', 'images (548).jpeg', 'images (489).jpeg', 'images (102).jpeg', 'images (293).jpeg', '

In [42]:
#https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison

shuffle = np.arange(pets.shape[0])
np.random.shuffle(shuffle)
pets = pets[shuffle]
labels = labels[shuffle]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(pets, labels, test_size=0.2, random_state=6)



In [44]:
y_train=keras.utils.to_categorical(y_train,2)
y_test=keras.utils.to_categorical(y_test,2)

In [45]:
#https://towardsdatascience.com/wtf-is-image-classification-8e78a8235acb
#https://www.tensorflow.org/tutorials/images/cnn

model = Sequential()
model.add(Conv2D(16,kernel_size=(3,3),activation = 'relu', input_shape = (50,50,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(64,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.summary()



Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 48, 48, 16)        160       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 24, 24, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 22, 22, 32)        4640      
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 11, 11, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 9, 9, 64)          18496     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 4, 4, 64)        

In [46]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
model.fit(X_train,y_train,batch_size=50,epochs=50,validation_data=(X_test, y_test))

Epoch 1/50
17/17 [==============================] - 3s 116ms/step - loss: 6.3769 - accuracy: 0.5085 - val_loss: 0.7897 - val_accuracy: 0.4904
Epoch 2/50
17/17 [==============================] - 2s 104ms/step - loss: 0.7557 - accuracy: 0.5399 - val_loss: 0.7113 - val_accuracy: 0.5337
Epoch 3/50
17/17 [==============================] - 2s 103ms/step - loss: 0.6676 - accuracy: 0.5966 - val_loss: 0.6908 - val_accuracy: 0.5481
Epoch 4/50
17/17 [==============================] - 2s 102ms/step - loss: 0.6290 - accuracy: 0.6280 - val_loss: 0.6900 - val_accuracy: 0.5529
Epoch 5/50
17/17 [==============================] - 2s 102ms/step - loss: 0.5921 - accuracy: 0.6800 - val_loss: 0.7189 - val_accuracy: 0.5529
Epoch 6/50
17/17 [==============================] - 2s 102ms/step - loss: 0.5550 - accuracy: 0.7017 - val_loss: 0.6988 - val_accuracy: 0.6010
Epoch 7/50
17/17 [==============================] - 2s 103ms/step - loss: 0.4878 - accuracy: 0.7657 - val_loss: 0.7843 - val_accuracy: 0.5817
Epoch 

In [48]:
score = model.evaluate(X_test, y_test, verbose=1)
print('\n', 'Test accuracy:', score[1])

7/7 [==============================] - 0s 20ms/step - loss: 1.7974 - accuracy: 0.6202

 Test accuracy: 0.620192289352417


In [49]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")